In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.de.load.loader import DataLoader
from src.common.dataprep import get_cleaned_data

In [3]:
# df = pd.read_csv('data/raw/bangkok_traffy.csv')
# shape = gpd.read_file('data/raw/BMA/BMA_ADMIN_SUB_DISTRICT.shp')

df_path = 'data/raw/bangkok_traffy.csv'
shape_path = 'data/raw/BMA/BMA_ADMIN_SUB_DISTRICT.shp'
df = get_cleaned_data(df_path, shape_path)

df.info()

c:\Users\Rirhc\OneDrive\Desktop\My Folder\CEDT\Year 2\Semester 1.2568\[DSDE]\Project with Git\JekTurnRight_dsde\src\common\dataprep.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = pd.to_datetime(df['timestamp'], format='ISO8601')
c:\Users\Rirhc\OneDrive\Desktop\My Folder\CEDT\Year 2\Semester 1.2568\[DSDE]\Project with Git\JekTurnRight_dsde\src\common\dataprep.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_activity'] = pd.to_datetime(df['last_activity'], format='ISO8601'

<class 'pandas.core.frame.DataFrame'>
Index: 510414 entries, 19 to 774306
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   ticket_id        510414 non-null  object             
 1   type             510414 non-null  object             
 2   organization     510414 non-null  object             
 3   comment          510414 non-null  object             
 4   coords           510414 non-null  object             
 5   address          510414 non-null  object             
 6   subdistrict      510414 non-null  object             
 7   district         510414 non-null  object             
 8   province         510414 non-null  object             
 9   timestamp        510414 non-null  datetime64[ns, UTC]
 10  state            510414 non-null  object             
 11  count_reopen     510414 non-null  int64              
 12  last_activity    510414 non-null  datetime64[ns, UTC]
 13  yea

In [6]:
df.head(10)

,ticket_id,type,organization,comment,coords,address,subdistrict,district,province,timestamp,state,count_reopen,last_activity,year_timestamp,month_timestamp,days_timestamp,longitude,latitude
19,2022-GNTHRA,{สะพาน},"เขตสาทร,ฝ่ายเทศกิจ เขตสาทร",ริมถนน ตรงสะพานสามช่วงจันทน์ 18/6-18/4 ชอบมีรถ...,"100.52760,13.70615",3 ซอย จันทน์ 18/4 แขวง ทุ่งวัดดอน เขต สาทร กรุ...,ทุ่งวัดดอน,สาทร,กรุงเทพมหานคร,2022-01-16 11:08:54.141813+00:00,เสร็จสิ้น,0,2022-06-23 08:06:52.928142+00:00,2022,1,31,100.52760,13.70615
20,2022-9EHJW2,"{จราจร,สะพาน,ถนน}","เขตสาทร,สน.ยานนาวา,ฝ่ายเทศกิจ เขตสาทร",บริเวณสะพานสามถนนจันทน์ เป็นจุดเปลี่ยนถ่ายสองแ...,"100.52756,13.70611",3 ซอย จันทน์ 18/4 แขวง ทุ่งวัดดอน เขต สาทร กรุ...,ทุ่งวัดดอน,สาทร,กรุงเทพมหานคร,2022-01-16 11:14:00.944767+00:00,เสร็จสิ้น,0,2022-06-22 08:18:08.293672+00:00,2022,1,31,100.52756,13.70611
25,2022-9KMF88,{ทางเท้า},"เขตสาทร,ฝ่ายเทศกิจ เขตสาทร",ซอยจันทน์ 16 ซอยขนาด 4 เลนแต่เลนติดบ้านคนกลายเ...,"100.53212,13.70578",14 ซ. จันทน์ 16 แขวง ทุ่งวัดดอน เขต สาทร กรุงเ...,ทุ่งวัดดอน,สาทร,กรุงเทพมหานคร,2022-01-17 03:01:52.819295+00:00,เสร็จสิ้น,0,2022-06-22 08:12:25.289345+00:00,2022,1,31,100.53212,13.70578
29,2022-39BTEG,{ถนน},"เขตลาดพร้าว,สำนักการจราจรและขนส่ง กรุงเทพมหานค...",ส่งรูปทางม้าลายไม่ชัดเจน และรถขายของเพิ่มเติม ...,"100.61849,13.82484",129 ถ. สุคนธสวัสดิ์ แขวงลาดพร้าว เขตลาดพร้าว ก...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2022-01-26 13:32:37.177987+00:00,เสร็จสิ้น,0,2024-11-13 06:06:16.579351+00:00,2022,1,31,100.61849,13.82484
32,2022-E449PD,{ถนน},"เขตประเวศ,สำนักการจราจรและขนส่ง กรุงเทพมหานคร ...",เส้นทางม้าลายไม่ชัด ตรงหน้าทางเข้าตลาดนัดรถไฟ ...,"100.64730,13.69141",13/131 ถนน ศรีนครินทร์ แขวง หนองบอน เขต ประเวศ...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2022-01-30 15:43:46.155790+00:00,เสร็จสิ้น,0,2022-06-23 23:48:29.379950+00:00,2022,1,31,100.64730,13.69141
34,2022-62HXGH,{ถนน},"เขตสาทร,สำนักการจราจรและขนส่ง กรุงเทพมหานคร (ส...",เรื่องทางม้าลายหายไป บริเวณสี่แยกถนนจันทร์-เซน...,"100.52680,13.70629",1878 ซอย จันทน์ 18/8 แขวง ทุ่งวัดดอน เขต สาทร ...,ทุ่งวัดดอน,สาทร,กรุงเทพมหานคร,2022-02-01 18:11:20.963592+00:00,เสร็จสิ้น,0,2023-02-10 09:48:30.498845+00:00,2022,2,28,100.52680,13.70629
37,2022-C9AX76,{แสงสว่าง},"กรมป้องกันและบรรเทาสาธารณภัย,ศูนย์อำนวยการบรรเ...",ไม่มีไฟส่องสว่าง #ทางม้าลาย,"100.50906,13.77239",12/1 ถนน อู่ทองนอก แขวง ดุสิต เขตดุสิต กรุงเทพ...,ดุสิต,ดุสิต,กรุงเทพมหานคร,2022-02-04 04:57:37.136868+00:00,เสร็จสิ้น,0,2024-03-05 11:51:11.518940+00:00,2022,2,28,100.50906,13.77239
41,2022-GMVHXT,{น้ำท่วม},"เขตสาทร,สำนักการระบายน้ำ กทม.,กองระบบท่อระบายน...",ปัญหาน้ำท่วมในซอยสวนพลู 1,"100.53882,13.71880",483 ถ. สวนพลู แขวง ทุ่งมหาเมฆ เขต สาทร กรุงเทพ...,ทุ่งมหาเมฆ,สาทร,กรุงเทพมหานคร,2022-02-04 12:06:06.193756+00:00,เสร็จสิ้น,0,2022-11-13 05:55:26.387723+00:00,2022,2,28,100.53882,13.71880
44,2022-FX8UWF,{เสียงรบกวน},"เขตลาดพร้าว,ฝ่ายสิ่งแวดล้อมฯ เขตลาดพร้าว",เเจ้งเรื่อง เสียงดังรบกวน\nร้านวนิดา ขายของมือ...,"100.59393,13.82443",1/173 หมู่ 10 ถนนโชคชัย 4 แขวงลาดพร้าว เขตลาดพ...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2022-02-07 00:15:45.797321+00:00,เสร็จสิ้น,0,2022-12-01 09:39:56.224334+00:00,2022,2,28,100.59393,13.82443
49,2022-9PH3BZ,"{ทางเท้า,สายไฟ}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",1. ซ่อมสายไฟ กรีดขวางทางเท้า,"100.65286,13.71860",31/1 ถ. อ่อนนุช แขวง ประเวศ เขต ประเวศ กรุงเทพ...,ประเวศ,ประเวศ,กรุงเทพมหานคร,2022-02-07 06:14:45.586614+00:00,เสร็จสิ้น,0,2022-06-29 08:47:22.964827+00:00,2022,2,28,100.65286,13.71860
